In [1]:
require 'nngraph'
require 'nn'

In [2]:
-- Try out nn.CAddTable which does element-wise addition

add = nn.CAddTable()
t1 = torch.Tensor({1,2,3})
t2 = torch.Tensor({3,4,10})
print(add:forward({t1, t2}))

  4
  6
 13
[torch.DoubleTensor of size 3]



In [3]:
-- Regarding the gradients of CAddTable
-- For this layer, we only have a gradient with respect to the input. There are no parameters of this layer.
-- Since we are just adding, dLoss/dinput_i = dLoss/dOutput for all of the input tensors of this layer

In [9]:
--nngraph overloads the parenthesis, "()", operator on all nn.Module objects
-- So we can rewrite the previous addition example using this notation

x1 = nn.Identity()() -- first () specify no arguments. second () specify input to module. Blank input indicates that it is a "dummy input", ie input of entire network
x2 = nn.Identity()()
a = nn.CAddTable()({x1, x2})
m = nn.gModule({x1, x2}, {a})

-- Note. nn.Identity() is an object of class nn.Module. But nn.Identity()() is a graph node
-- nn.gModule wraps the graph into a single object, which we can use as a normal nn.Module()

t1 = torch.Tensor({10, 20, 30})
t2 = torch.Tensor({50, 10, 20})
print(m:forward({t1, t2}))



 60
 30
 50
[torch.DoubleTensor of size 3]



In [21]:
in1 = torch.ones(5) * 2
in2 = torch.ones(5) * 4
in3 = torch.range(1,10)

x1 = nn.Identity()()
x2 = nn.Identity()()
x3 = nn.Identity()()

linear = nn.Linear(10, 5)({x3})
weights = torch.ones( linear.data.module.weight:size() ) -- initialise the linear weights to 1 so they are easy to debug
linear.data.module.weight = weights
linear.data.module.bias = torch.range(1,5)

print('Linear Weights')
print(linear.data.module.weight)

c1 = nn.CMulTable()({x2, linear})
z = nn.CAddTable()({x1, c1})

net = nn.gModule({x1, x2, x3}, {z})

print("Output of network")
print(net:forward({in1, in2, in3}))
print("\nOutput of linear layer")
print(linear.data.module.output)

Linear Weights	
 1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1
[torch.DoubleTensor of size 5x10]

Output of network	
 226
 230
 234
 238
 242
[torch.DoubleTensor of size 5]


Output of linear layer	
 56
 57
 58
 59
 60
[torch.DoubleTensor of size 5]

